### Mac OS Protobuf Error Fixing Code Snipet

In [4]:
!pip install protobuf==3.19.4



  Using cached protobuf-3.19.4-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.4 which is incompatible.


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf


objc[49627]: Class CaptureDelegate is implemented in both /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15b1524e8) and /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x12a2e0860). One of the two will be used. Which one is undefined.
objc[49627]: Class CVWindow is implemented in both /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15b152538) and /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x10a848a68). One of the two will be used. Which one is undefined.
objc[49627]: Class CVView is implemented in both /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15b152560) and /Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x10a848a90). One of the two will be used. Which

NotFoundError: dlopen(/Users/hwangjaeyeong/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '__ZN10tensorflow8internal10LogMessage16VmoduleActivatedEPKci'

In [2]:
!brew install python@3.11

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################## 100.0%
To reinstall 3.11.2_1, run:
  brew reinstall python@3.11


In [2]:
!brew reinstall protobuf

==> Fetching protobuf
==> Downloading https://ghcr.io/v2/homebrew/core/protobuf/manifests/21.12-2
Already downloaded: /Users/hwangjaeyeong/Library/Caches/Homebrew/downloads/dc99692625dd520614284dd5dd5be130c983c7ce0e3de360c51595589b6c1a49--protobuf-21.12-2.bottle_manifest.json
==> Downloading https://ghcr.io/v2/homebrew/core/protobuf/blobs/sha256:09fdc2fcc
Already downloaded: /Users/hwangjaeyeong/Library/Caches/Homebrew/downloads/b7cc6c6edab103991821f556b01bb53d1e251427e20d13b3456bd02734aea989--protobuf--21.12.arm64_ventura.bottle.2.tar.gz
==> Reinstalling protobuf 
==> Pouring protobuf--21.12.arm64_ventura.bottle.2.tar.gz
🍺  /opt/homebrew/Cellar/protobuf/21.12: 288 files, 19.2MB
==> Running `brew cleanup protobuf`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


In [7]:
!python -m pip install tensorflow-metal

In [6]:
!python -m pip install tensorflow-macos

  Using cached protobuf-3.19.6-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.


In [1]:

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="/Volumes/T7/Project_Database/2023_Google_Solution_Challenge/DL-GoldenHour/wound_detection/wound_detection_model/best_metadata.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Define function to find nearest landmark to a point
def find_nearest_landmark(point, landmarks):
    distances = np.linalg.norm(landmarks - point, axis=1)
    nearest_idx = np.argmin(distances)
    return landmarks[nearest_idx]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Run MediaPipe Pose model to get landmarks
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    landmarks = np.array([[landmark.x, landmark.y] for landmark in results.pose_landmarks.landmark])

    # Run TFLite model to predict wound location and name
    input_data = np.expand_dims(frame, axis=0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    wound_location = output_data[0][:2]
    wound_name = output_data[0][2]

    # Draw red bounding box around wound location
    x, y = map(int, wound_location)
    cv2.rectangle(frame, (x-20, y-20), (x+20, y+20), (0, 0, 255), 2)

    # Find 2 landmarks near wound location
    nearest_landmarks = np.array([
        find_nearest_landmark(wound_location, landmarks),
        find_nearest_landmark(wound_location + np.array([0, -50]), landmarks)
    ])

    # If wound is a bite wound, find nearest landmark near shoulder or hip
    if wound_name == 'wound_ulcer':
        candidate_landmarks = np.array([
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks)
        ])
        nearest_landmarks[1] = find_nearest_landmark(nearest_landmarks[0], candidate_landmarks)

    # Calculate midpoint between wound location and nearest landmark
    midpoint = np.mean(nearest_landmarks, axis=0)

    # Draw red circle at midpoint
    cv2.circle(frame, tuple(map(int, midpoint)), 10, (0, 0, 255), -1)

    # Show frame
    cv2.imshow('Frame', frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


NameError: name 'tf' is not defined